# Web Scraper Booster
If you're in this group then you already have a bit of a head start because you've already seen and used the requests library.  You'll be drawing on this library again, combining it with the powerful parser known as Beautiful Soup to extract the content you desire.

There is some good discussion of `requests` at https://medium.com/@anthonypjshaw/python-requests-deep-dive-a0a5c5c1e093.  Pay special attention to the note about keeping the requests session alive.

There is an example of how to use requests to get content from Gutenberg (which you have already seen a bit of) and then use Beautiful Soup to process it.

##### Astrid
Having some trouble seeing where to get the content from pages like http://sounds.bl.uk/Accents-and-dialects/BBC-Voices/021M-C1190X0043XX-0101V0 since they use javascript to control where/when the content is displayed (effectively hiding the actual URL and preventing downloads).  There should be a way to see what the actual location of the sound file is programmatically.  I can do it using the Chrome developer tools and watching the network stream to get the actual URL for the data content:  http://194.66.233.31/SoundsServer2016/BBC-Voices/021A-C1190X0043XX-0101A0.mp3?dk=381A09163EE7321C94EA717B240D024E2E1C92FE60E8F2B7481B10444F064E9221AD9DEF287AA791364FB0116B8FB583EE4CDD688D01E854BA371626F2B4372167319E76D48E04E2 . I have not found a way to do this directly because of the long "key" (this is everything after the `?`) that is required to access the content.  Effectively what is being worked with here is an API that requires permissions.  Given the setup there will be a way to trick the server into giving us the needed URLs but given the time it might take to figure this out by picking through their javascript and guessing at the mechanisms it might be better to just grab the URLs via Chrome and then use Python to process them all.

Found a library that we could likely use for YouTube content: https://pypi.python.org/pypi/pafy 

### Use `requests` to grab content of a single web page

In [ ]:
#Just grab the content of a single book
import requests

response=requests.get("http://www.gutenberg.org/files/2701/2701-h/2701-h.htm")
print(response.text)

Notice that the output of the above cell is different from the plain/raw text content we had before.  Now we have HTML tags marking up the text.  Consider the following before moving on:

1. What happens if you use `print(response)` instead of `print(response.text)`?
2. What other methods can be used with `request` objects? Use `print()` to see what they are

### Reading from a list of urls and looping
Create a plain text file that has other HTML file URLs from the Gutenberg website (you'll need to just use your browser for now to navigate to each page and copy the URL from the address box).  Note the following requirements:

1. Each URL must have its own line.
2. One of the URLs _should_ be: http://www.gutenberg.org/files/2591/2591-h/2591-h.htm

Then use the following code as a guide to load the content of each page into a list where each member is the page content of each URL.

In [ ]:
works=[]
with open("files.txt",'r') as files:
    for line in files:
        response=requests.get(line)
        works.append(response.text)

### Using Beautiful Soup
Time to do some soupification.  Correct and play around with the following to see what you can get out of the content of each page.  Use your browser to _really look_ at the HTML content of the pages you have captured and choose elements that you might want to capture and then try to capture them.

In [ ]:
from bs4 import BeautifulSoup

soup=BeautifulSoup(works[PUT THE INDEX OF THE ITEM HERE],"lxml")
h2s=soup.find_all('h2')
#print(h2s)
#print(type(h2s[1]))
for h2 in h2s:
    print(h2.text.strip())

Before moving on make sure that "soup" has been set to the GRIMM FAIRYTALES.

### Some tricky content grabbing
It is nice to work with web pages where all the content is wrapped up in a nice hierarchy of tags.  The Gutenberg content does not do this and so other than making simple calls like "get me all the `<a>` tags" there is some work to do to grab other content.  For example, if we wanted to grab all the content from the chapter called "The Raven" we'd need to do something like the following.

In [ ]:
for h2 in h2s:
    if h2.text.strip() == "THE RAVEN":
        # sibling = h2.nextSibling
        # closingH2 = sibling.findNext("h2")
        # print (sibling.next_sibling)
        # print (sibling.next_sibling.next_sibling)
        # print (sibling.next_sibling.next_sibling.next_sibling)
        nextElement = h2.nextSibling.findAllNext()
        nextA = h2.nextSibling.findNext('a')
        #print (nextA)
        story = []
        for element in nextElement:
            if element == nextA:
                break
            #print (element.text)
            story.append(element.text)
#        print(story)
#        print (sibling.next_element.next_element.next_element.next_element)
#        print (sibling, closingH2)
#        while sibling != closingH2:
#            print (sibling.next_element)

What are all the comments in the code above?  They are left over from the figuring out of how to grab just a chapter. What you need to to is read and run the code a few times to figure out what it is doing at each step.  Uncommenting parts of the code to see what they reveal should be helpful.

### Tokenizing
Each of you will want to do something different with the page content that you end up scraping but you might as well see how to tokenize it so here's an example of that.  Note that the characters used in Grimm stories cause trouble for the tokenizer and there is a workaround built into the content below.  Can you see what the workaround is and disable it to see the original problem?

Note that this bit of code will require you to run the Natural Language ToolKit (`nltk`).  It is likely that you will get an error the first time you run this because your instance will be missing something.  _READ_ the error.  Note that it is likely that trying to solve the error in Jupyter Notebooks will crash your notebook server so I recommend going to the command line to do the solution.  Here are two hints:

1. `$ python`
2. When you pass the suggested code to the interpreter a window will pop-up.  It is very likely that it will do so _behind_ your other windows.

In [ ]:
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
import string

#allStoryText = word_tokenize(" ".join(story))
#print(allStoryText)
fullStory = " ".join(story)
fullStory = fullStory.replace('‘','"')
fullStory = fullStory.replace('’','"')
tokenStory = word_tokenize(fullStory)
print(tokenStory)

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ["''","``"] 
# Last two because of how tokenizing with NLTK converts " to '' and ``
#print(stop)

terms = [term.lower() for term in tokenStory if term.lower() not in stop]
#print(terms)

count_all = Counter()
count_all.update(terms)
#print(count_all)

### Getting ready to plot
You now have all the information that you need to plot.  Before you do that though you should play with code below by changing the values and commenting out some of the lines to figure out what it is doing/how it works.

In [ ]:
import matplotlib.pyplot as plt

#special ipython/jupyter command that keeps the output in this window rather than opening another one.
%matplotlib inline 

plt.figure()
plt.plot([1, 2, 3, 4], [10, 20, 25, 30], color='lightblue', linewidth=3)
plt.scatter([0.3, 3.8, 1.2, 2.5], [11, 25, 9, 26], color='darkgreen', marker='^')
plt.bar([1,2,3,4],[12,3,25,18], width=0.2, align='center')
plt.xlim(0.5, 4.5)
plt.ylim(0,50)
plt.show()

### Plotting the story words
The code below will plot the ten most frequent words from the story we grabbed.  Again, you should be commenting lines out and changing values to see what changes.  Compare the layout to the simpler example above.  Make sure that you can _describe_ what is happening and why at each line.

In [ ]:
%matplotlib inline
count_all_dict = dict(count_all.most_common(10))
import matplotlib.pyplot as plt
#plt.figure(figsize=(100, 40))
plt.bar(range(len(count_all_dict)), count_all_dict.values(), align='center')
plt.xticks(range(len(count_all_dict)), list(count_all_dict.keys()),rotation='vertical')

plt.show()